Este notebook nos dará los casos posticios y los casos negativos para las vacantes

In [1]:
import pandas as pd 
import numpy as np

In [30]:
vacants = pd.read_json("../files/raw/vacants.json")
applications = pd.read_json("../files/sample_for_capacity/applications.json")
candidates = pd.read_json("../files/sample_for_capacity/candidates.json")


In [31]:
print(len(vacants), len(candidates), len(applications))

142728 180539 2461188


In [32]:
vacants["vacant_id"].value_counts().head(565)

vacant_id
522841    1
57283     1
92879     1
155066    1
159462    1
         ..
242721    1
242720    1
242719    1
242718    1
242717    1
Name: count, Length: 565, dtype: int64

In [6]:
applications = applications.copy()
vacants = vacants.copy()

applications["created_at"] = pd.to_datetime(applications["created_at"], errors="coerce", unit="ms", utc=True)
vacants["publish_date"] = pd.to_datetime(vacants["publish_date"], errors="coerce", unit="ms", utc=True)
apps = applications[["candidate_id","vacant_id","created_at","stage_order", "application_status"]].dropna(subset=["candidate_id","vacant_id"])


In [33]:
g = apps.groupby(['candidate_id','vacant_id'], as_index=False)

t_apply = g['created_at'].min().rename(columns={'created_at':'t_apply'})
stage_max = g['stage_order'].max().rename(columns={'stage_order':'stage_max'})

# t_pos = (apps.loc[apps['stage_order'] > 1]
#              .groupby(['candidate_id','vacant_id'], as_index=False)['created_at']
#              .min()
#              .rename(columns={'created_at':'t_pos'}))

# t_reject = (apps.loc[(apps['stage_order']==1) & (apps['application_status']==1)]
#                 .groupby(['candidate_id','vacant_id'], as_index=False)['created_at']
#                 .min()
#                 .rename(columns={'created_at':'t_reject'}))

pairs = (t_apply
         .merge(stage_max, on=['candidate_id','vacant_id'], how='left')
         .merge(vacants[['vacant_id','publish_date']], on='vacant_id', how='left')
        )
pairs

,candidate_id,vacant_id,t_apply,stage_max,publish_date
0,91,245572,2024-02-15 18:32:30.977000+00:00,1.0,1705668796588
1,91,246506,2024-02-15 18:38:06.610000+00:00,2.0,1706020679844
2,91,247638,2024-02-15 18:55:04.656000+00:00,1.0,1706292727786
3,91,248048,2024-02-15 18:32:14.009000+00:00,1.0,1706543845062
4,91,248749,2024-02-15 18:44:36.013000+00:00,1.0,1706707271582
...,...,...,...,...,...
2460409,8850826,522454,2025-05-31 23:49:12.386000+00:00,1.0,1748641351685
2460410,8850836,509332,2025-07-16 18:32:39.021000+00:00,1.0,1747177244526
2460411,8850852,500757,2025-07-02 14:04:16.843000+00:00,1.0,1745935719401
2460412,8850852,521689,2025-07-03 04:39:35.609000+00:00,1.0,1748561150354


In [34]:
pairs['label'] = (pairs['stage_max'] > 1).astype(int)
pairs

,candidate_id,vacant_id,t_apply,stage_max,publish_date,label
0,91,245572,2024-02-15 18:32:30.977000+00:00,1.0,1705668796588,0
1,91,246506,2024-02-15 18:38:06.610000+00:00,2.0,1706020679844,1
2,91,247638,2024-02-15 18:55:04.656000+00:00,1.0,1706292727786,0
3,91,248048,2024-02-15 18:32:14.009000+00:00,1.0,1706543845062,0
4,91,248749,2024-02-15 18:44:36.013000+00:00,1.0,1706707271582,0
...,...,...,...,...,...,...
2460409,8850826,522454,2025-05-31 23:49:12.386000+00:00,1.0,1748641351685,0
2460410,8850836,509332,2025-07-16 18:32:39.021000+00:00,1.0,1747177244526,0
2460411,8850852,500757,2025-07-02 14:04:16.843000+00:00,1.0,1745935719401,0
2460412,8850852,521689,2025-07-03 04:39:35.609000+00:00,1.0,1748561150354,0


In [35]:
pairs["label"].value_counts()

label
0    1955344
1     505070
Name: count, dtype: int64

In [36]:
# H_NEG_DAYS = 30
# FAST_NEG_DAYS = 7  


# pairs.loc[pairs['t_label_known'].isna(), 't_label_known'] = pairs['t_apply'] + pd.Timedelta(days=H_NEG_DAYS)
# pairs['is_quick_neg'] = (
#     (pairs['label'] == 0) &
#     pairs['t_reject'].notna() &
#     ((pairs['t_reject'] - pairs['t_apply']) <= pd.Timedelta(days=FAST_NEG_DAYS))
# )

In [37]:
T_TRAIN_END = pd.Timestamp('2024-12-31', tz='UTC')  # Ajusta si quieres
T_VAL_END   = pd.Timestamp('2025-05-31', tz='UTC')  # Ajusta si quieres
DELTA_DAYS  = 14
DELTA = pd.Timedelta(days=DELTA_DAYS)


In [38]:
train = pairs.loc[pairs['t_apply'] <= T_TRAIN_END].copy()
val   = pairs.loc[(pairs['t_apply'] > T_TRAIN_END + DELTA) & (pairs['t_apply'] <= T_VAL_END)].copy()
test  = pairs.loc[pairs['t_apply'] > T_VAL_END + DELTA].copy()

In [39]:
C_train = set(train['candidate_id'].unique())
V_train = set(train['vacant_id'].unique())

def tag_scenario(df, C_seen, V_seen):
    c_new = ~df['candidate_id'].isin(C_seen)
    v_new = ~df['vacant_id'].isin(V_seen)
    return np.select(
        [~c_new & ~v_new,  c_new & ~v_new, ~c_new & v_new,  c_new & v_new],
        ['warm',          'cold_c',       'cold_v',        'zero'],
        default='warm'
    )

val['scenario']  = tag_scenario(val,  C_train, V_train)
test['scenario'] = tag_scenario(test, C_train, V_train)

# Disjoint por par entre splits
k = ['candidate_id','vacant_id']
assert set(map(tuple, train[k].to_numpy())).isdisjoint(set(map(tuple, val[k].to_numpy())))
assert set(map(tuple, train[k].to_numpy())).isdisjoint(set(map(tuple, test[k].to_numpy())))
assert set(map(tuple, val[k].to_numpy())).isdisjoint(set(map(tuple, test[k].to_numpy())))

# Reglas de escenarios
assert set(val.loc[val['scenario']=='cold_c','candidate_id']).isdisjoint(C_train)
assert set(val.loc[val['scenario']=='cold_v','vacant_id']).isdisjoint(V_train)
assert set(test.loc[test['scenario']=='cold_c','candidate_id']).isdisjoint(C_train)
assert set(test.loc[test['scenario']=='cold_v','vacant_id']).isdisjoint(V_train)

In [40]:
def summarize(df, name):
    return {
        'split': name,
        'rows': len(df),
        'positivos': int((df['label']==1).sum()),
        'negativos': int((df['label']==0).sum()),
        'candidatos': df['candidate_id'].nunique(),
        'vacantes': df['vacant_id'].nunique(),
        't_min': df['t_apply'].min(),
        't_max': df['t_apply'].max()
    }

summary = pd.DataFrame([summarize(train,'train'),
                        summarize(val,'val'),
                        summarize(test,'test')])
print(summary)
print("\nValidación por escenario:")
print(val.groupby('scenario')['label'].agg(rows='size', pos=lambda s: int((s==1).sum())))
print("\nTest por escenario:")
print(test.groupby('scenario')['label'].agg(rows='size', pos=lambda s: int((s==1).sum())))


   split     rows  positivos  negativos  candidatos  vacantes  \
0  train  1447128     314467    1132661      127135     69238   
1    val   860483     159041     701442       92086     47636   
2   test    43884       9681      34203       22370      5825   

                             t_min                            t_max  
0 2024-01-01 21:04:45.443000+00:00 2024-12-30 23:59:33.016000+00:00  
1 2025-01-14 00:00:00.999000+00:00 2025-05-30 23:59:45.170000+00:00  
2 2025-06-14 00:03:45.256000+00:00 2025-11-25 23:20:51.654000+00:00  

Validación por escenario:
            rows    pos
scenario               
cold_c     23228   9289
cold_v    489158  84544
warm       29181   8757
zero      318916  56451

Test por escenario:
           rows   pos
scenario             
cold_c      642   120
cold_v    22399  4819
warm        665   148
zero      20178  4594


In [41]:
test

,candidate_id,vacant_id,t_apply,stage_max,publish_date,label,scenario
137,319,502825,2025-06-24 19:54:11.760000+00:00,1.0,1746204569247,0,cold_v
138,319,504093,2025-06-17 15:08:55.866000+00:00,1.0,1746467371271,0,cold_v
139,319,506062,2025-09-30 16:26:07.138000+00:00,1.0,1746647241149,0,cold_v
140,319,509988,2025-10-07 12:27:42.682000+00:00,1.0,1747765739580,0,cold_v
141,319,511464,2025-06-23 22:34:31.797000+00:00,1.0,1747328590510,0,cold_v
...,...,...,...,...,...,...,...
2460402,8850776,505026,2025-07-10 05:03:45.486000+00:00,1.0,1746543119326,0,zero
2460403,8850776,512628,2025-07-10 05:03:47.859000+00:00,1.0,1747435983454,0,zero
2460410,8850836,509332,2025-07-16 18:32:39.021000+00:00,1.0,1747177244526,0,zero
2460411,8850852,500757,2025-07-02 14:04:16.843000+00:00,1.0,1745935719401,0,zero


In [42]:
v=vacants[["city_loc","full_text", "vacant_id", "city_ids", "remote"]].rename(columns={"full_text":"vacant_full_text", "city_loc":"vacant_city_loc","city_ids":"vacant_city_ids", "remote":"vacant_remote"})
train=train.merge(v, on="vacant_id")
test=test.merge(v, on="vacant_id")
val=val.merge(v, on="vacant_id")


In [43]:
c=candidates[["candidate_id", "full_text", "city_loc", "city_id"]].rename(columns={"full_text":"candidate_full_text", "city_loc":"candidate_city_loc", "city_id":"candidate_city_id"})

In [ ]:
train

In [46]:

train=train.merge(c, on="candidate_id")
test=test.merge(c, on="candidate_id")
val=val.merge(c, on="candidate_id")


In [48]:
train.to_parquet("../files/processed/train.parquet", index=False)
val.to_parquet("../files/processed/val.parquet", index=False)
test.to_parquet("../files/processed/test.parquet", index=False)

In [47]:
train.dtypes

candidate_id                         int64
vacant_id                            int64
t_apply                datetime64[ns, UTC]
stage_max                          float64
publish_date                         int64
label                                int64
vacant_city_loc                     object
vacant_full_text                    object
vacant_city_ids                     object
vacant_remote                         bool
candidate_full_text                 object
candidate_city_loc                  object
candidate_city_id                    int64
dtype: object